In [1]:
from typing import Dict
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import ellipj, ellipkinc, ellipeinc, ellipk

In [2]:
def bisection_method(func, _r,_alpha, bh_mass, incl, n, a, b, tol):
    if func(a, _r,_alpha, bh_mass, incl, n) * func(b, _r,_alpha, bh_mass, incl, n) >= 0:
        print("Bisection method fails. The function must have opposite signs at a and b.")
        return None

    while (b - a) / 2 > tol:
        midpoint = (a + b) / 2
        if func(midpoint, _r,_alpha, bh_mass, incl, n,) == 0:  # Found exact root
            return midpoint
        elif func(a, _r,_alpha, bh_mass, incl, n) * func(midpoint, _r,_alpha, bh_mass, incl, n) < 0:
            b = midpoint
        else:
            a = midpoint

    return (a + b) / 2

def find_multiple_roots(func, _r,_alpha, bh_mass, incl, n, start, end, step, tol=1e-6):
    roots = []
    x = start
    while x < end:
        x_next = x + step
        if func(x, _r,_alpha, bh_mass, incl, n) * func(x_next, _r,_alpha, bh_mass, incl, n) < 0:  # Root exists in [x, x_next]
            try:
                root = bisection_method(func, _r,_alpha, bh_mass, incl, n, x, x_next, tol)
                roots.append(root)
            except ValueError:
                pass  # Skip intervals where the method fails
        x = x_next
    return roots

In [3]:
def calcQ(P: float,bh_mass: float, tol=1e-3) ->float:
    Q=np.sqrt((P - 2.*bh_mass)*(P + 6.*bh_mass))
    return Q

def bfromP(P: float,bh_mass: float, tol: float=1e-5) ->float:
    b = np.sqrt(P**3/(P - 2.*bh_mass))
    return b

def zetainf(P: float,bh_mass: float, tol=1e-6) ->float:
    Q = calcQ(P,bh_mass)
    arg =(Q-P+2.*bh_mass)/(Q-P + 6.*bh_mass)
    z_inf = np.arcsin(np.sqrt(arg))
    return z_inf

def k(P: float,bh_mass: float) ->float:
    Q = calcQ(P,bh_mass)
    k = np.sqrt((Q-P+6*bh_mass)/(2*Q))
    return k

def k2(P: float,bh_mass: float) ->float:
    Q = calcQ(P,bh_mass)
    k2 = (Q-P+6*bh_mass)/(2*Q)
    return k2

def cosgamma(_a: float, incl: float, tol=1e-5) -> float:
    if abs(incl) <tol:
        return 0
    return np.cos(_a)/np.sqrt(np.cos(_a)**2+1/(np.tan(incl)**2))

def eq13(P: float, ir_radius: float,ir_angle: float, bh_mass: float,incl: float, n: int=0, tol=10e-6) -> float:
    z_inf = zetainf(P,bh_mass)
    Q = calcQ(P,bh_mass)
    m_ = k2(P,bh_mass)
    ell_inf = ellipkinc(z_inf,m_)
    g = np.arccos(cosgamma(ir_angle, incl))
    if n:
        ell_k = ellipk(m_)
        ellips_arg = (g-2.*n*np.pi)/(2.*np.sqrt(P/Q))-ell_inf+2.*ell_k
    else:
        ellips_arg=g/(2.*np.sqrt(P/Q)) +ell_inf
    sn,cn,dn,ph = ellipj(ellips_arg, m_)
    sn2 = sn*sn
    term1 = -(Q-P+2.*bh_mass)/(4.*bh_mass*P)
    term2 =((Q-P+6*bh_mass)/(4.*bh_mass*P))*sn2
    return 1.-ir_radius*(term1+term2)

def calcP(_r, incl, _alpha, bh_mass, n=0, min_P=3.001, step=0.001) -> float:
    Periastrons = find_multiple_roots(eq13, _r, _alpha, bh_mass, incl, n, min_P, 2*_r, step, tol=1e-6)
    return Periastrons

def ellipse(r,a,incl):
    g=np.arccos(cosgamma(a,incl))
    b_=r*np.sin(g)
    return b_

def calcb(_r, incl, _alpha, bh_mass, n, min_P, step) -> float:
    periastrons = calcP(_r,incl,_alpha,bh_mass, n,min_P,step)
    for i in range(len(periastrons)):
        b=np.empty(len(periastrons))
        if periastrons[i] is None or periastrons[i] <=2.*bh_mass:
            b[i]= ellipse(_r, _alpha,incl)
        elif periastrons[i] >2.*bh_mass:
            b[i]=bfromP(periastrons[i], bh_mass)
        else:
            raise ValueError("no solution was found for P ar (r,a)=({_r}, {_alpha}) and incl = {incl}")
        return b

In [4]:
def redshifteq(radius, angle, incl, bh_mass, n, min_P, step, z):
    b= calcb(radius, incl, angle, bh_mass, n, min_P, step)
    if b is not None:
        return (1.+np.sqrt(bh_mass/radius**3)*b*np.sin(incl)*np.sin(angle))/np.sqrt(1-3.*bh_mass/radius)-(1+z)
    else:
        return None

def bisection_method_redshift(func, _alpha, incl, bh_mass, n, min_P, step, z, a, b, tol):
    if func(a, _alpha, incl, bh_mass, n, min_P, step, z) * func(b, _alpha, incl, bh_mass, n, min_P, step, z,) >= 0:
        print("Bisection method fails. The function must have opposite signs at a and b.")
        return None

    while (b - a) / 2 > tol:
        midpoint = (a + b) / 2
        if func(midpoint, _alpha, incl, bh_mass, n, min_P, step, z) == 0:  # Found exact root
            return midpoint
        elif func(a, _alpha, incl, bh_mass, n, min_P, step, z) * func(midpoint, _alpha, incl, bh_mass, n, min_P, step, z) < 0:
            b = midpoint
        else:
            a = midpoint

    return (a+b)/2

def find_multiple_roots_redshift(func, _alpha, bh_mass, incl, n, min_P, z, start, end, step, tol=1e-6):
    roots = []
    x = start
    while x < end:
        x_next = x + step
        if (func(x, _alpha, incl, bh_mass, n, min_P,step, z) is not None or func(x_next, _alpha, incl, bh_mass, n, min_P, step, z) is not None) and func(x, _alpha, incl, bh_mass, n, min_P, step, z) * func(x_next, _alpha, incl, bh_mass, n, min_P, step, z) < 0:  # Root exists in [x, x_next]
            try:
                root = bisection_method_redshift(func, _alpha, incl, bh_mass, n, min_P, step, z, x, x_next, tol)
                roots.append(root)
            except ValueError:
                pass  # Skip intervals where the method fails
        x = x_next
    return roots

def constantredshift(incl, _alpha, bh_mass, z, min_r, n=0, min_P=1., step=0.001) -> float:
    radii = find_multiple_roots_redshift(redshifteq, _alpha, bh_mass, incl, n, min_P, z, min_r, 60*bh_mass, step)
    return radii

In [ ]:
alphas = np.linspace(0, 2*np.pi, 1000)
constantz_085_30 = []
constantz_090_30 = []
constantz_095_30 = []
constantz_100_30 = []
constantz_105_30 = []
constantz_110_30 = []
constantz_115_30 = []
constantz_120_30 = []
constantz_125_30 = []
constantz_150_30 = []
constantz_175_30 = []

for i in range(0,1000):
    constantz_085_30.append(constantredshift(np.pi/3, alphas[i], 1, -0.15, 3.001, 0, 3.001, 0.001))
    constantz_090_30.append(constantredshift(np.pi/3, alphas[i], 1, -0.1, 3.001, 0, 3.001, 0.001))
    constantz_095_30.append(constantredshift(np.pi/3, alphas[i], 1, -0.05, 3.001, 0, 3.001, 0.001))
    constantz_100_30.append(constantredshift(np.pi/3, alphas[i], 1, 0, 3.001, 0, 3.001, 0.001))
    constantz_105_30.append(constantredshift(np.pi/3, alphas[i], 1, 0.05, 3.001, 0, 3.001, 0.001))
    constantz_110_30.append(constantredshift(np.pi/3, alphas[i], 1, 0.10, 3.001, 0, 3.001, 0.001))
    constantz_115_30.append(constantredshift(np.pi/3, alphas[i], 1, 0.15, 3.001, 0, 3.001, 0.001))
    constantz_120_30.append(constantredshift(np.pi/3, alphas[i], 1, 0.20, 3.001, 0, 3.001, 0.001))
    constantz_125_30.append(constantredshift(np.pi/3, alphas[i], 1, 0.25, 3.001, 0, 3.001, 0.001))
    constantz_150_30.append(constantredshift(np.pi/3, alphas[i], 1, 0.5, 3.001, 0, 3.001, 0.001))
    constantz_175_30.append(constantredshift(np.pi/3, alphas[i], 1, 0.75, 3.001, 0, 3.001, 0.001))

plt.figure(figsize=(6,6))
ax = plt.subplot(111,polar=True)
ax.plot(alphas, constantz_085_30, color= 'black', label="$z=-0.15$")
ax.plot(alphas, constantz_090_30, color= 'purple', label="$z=-0.10$")
ax.plot(alphas, constantz_095_30, color= 'blue', label="$z=-0.05$")
ax.plot(alphas, constantz_100_30, color= 'cyan', label="$z=0.00$")
ax.plot(alphas, constantz_105_30, color= 'olive', label="$z=0.05$")
ax.plot(alphas, constantz_110_30, color= 'green', label="$z=0.10$")
ax.plot(alphas, constantz_115_30, color= 'red', label="$z=0.15$")
ax.plot(alphas, constantz_120_30, color= 'orange', label="$z=0.20$")
ax.plot(alphas, constantz_125_30, color= 'brown', label="$z=0.25$")
ax.plot(alphas, constantz_150_30, color= 'pink', label="$z=0.50$")
ax.plot(alphas, constantz_175_30, color= 'grey', label="$z=0.75$")
plt.legend()
plt.show()